In [3]:
import tensorflow as tf 
import numpy as np
import pandas as pd
import cv2
import os


C:\Users\toumi\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [1]:
pwd

'C:\\Users\\toumi'

In [4]:
dir_names = []

for dirname, _, filenames in os.walk('C:\\Users\\toumi\\OneDrive\\Bureau\\ML files\\flowers classification\\flowers'):
    for filename in filenames:
        dir_name = os.path.join(dirname, filename)
        dir_names.append(dir_name)
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], dir_names))
dir_names = pd.Series(dir_names, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

# Concatenate filepaths and labels
image_df = pd.concat([dir_names, labels], axis=1)

In [ ]:
image_df.head(7)

In [5]:
from sklearn.model_selection import train_test_split
train,test = train_test_split( image_df,test_size=0.2, random_state=0,shuffle=True)
 


In [6]:
from tensorflow.keras.preprocessing.image import *
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator()

train_images = train_datagen.flow_from_dataframe(
    dataframe=train,
    x_col='Filepath',
    y_col='Label',
    target_size=(150, 150),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

Found 3453 validated image filenames belonging to 5 classes.


In [7]:
from tensorflow.keras.preprocessing.image import *
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator()

test_images = test_datagen.flow_from_dataframe(
    dataframe=test,
    x_col='Filepath',
    y_col='Label',
    target_size=(150, 150),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=100,
    subset='training'
)


Found 864 validated image filenames belonging to 5 classes.


In [8]:
from tensorflow.keras.layers  import Dense,InputLayer,Conv2D,MaxPool2D,Flatten, BatchNormalization, Dropout
model = tf.keras.Sequential([


#filters=6, kernal_size=5, strides=1, padding='valid', activation='sigmoid'


    tf.keras.layers.Conv2D( filters=6,
    kernel_size=5,
    strides=(1, 1),
    padding='valid',
    activation='sigmoid', input_shape = (150,150,3)),
    MaxPool2D(pool_size=2,strides=2),


    tf.keras.layers.Conv2D( filters=16,
    kernel_size=5,
    strides=(1, 1),
    padding='valid',
    activation='sigmoid',),
    BatchNormalization(),
    MaxPool2D(pool_size=2,strides=2),
    Dropout(0.2),

    Flatten(),

    Dense(100, activation='relu'),
    BatchNormalization(),
    Dense(10 , activation='relu'),
    Dropout(0.2),

    Dense(5 , activation='sigmoid'),

])
#model.summary()

In [9]:
#load last weights and note back from 0 
model.load_weights("C:\\Users\\toumi\\OneDrive\\Bureau\\rayen staff\\weights_best.hdf5")


In [10]:

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import RootMeanSquaredError
model.compile(optimizer= Adam(learning_rate=0.001),
              loss=CategoricalCrossentropy(),
              metrics=['accuracy'])

In [11]:
# checkpoint callback
filepath="C:\\Users\\toumi\\OneDrive\\Bureau\\rayen staff\\weights_best.hdf5"
checkpoint=tf.keras.callbacks.ModelCheckpoint(filepath, 
                                     monitor='val_accuracy', 
                                     verbose=0, 
                                     save_best_only=False,
                                     save_weights_only=False, 
                                     mode='max', 
                                     save_freq='epoch')
callbacks_list=[checkpoint]

In [12]:
#early stopping
early_stoping= tf.keras.callbacks.EarlyStopping(
    monitor='loss', min_delta=0, patience=2, verbose=1,
    mode='auto'
)

In [13]:
#csv logger
CSvlogger=tf.keras.callbacks.CSVLogger('training.log', 
                             separator=',', 
                             append=False)

In [14]:
def scheduler(epoch, lr):

  if epoch <= 1:
    learning_rate = lr
  else:
    learning_rate = lr * tf.math.exp(-0.1)
    learning_rate = learning_rate.numpy()


  return learning_rate
scheduler_callback = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose = 1)

In [ ]:
  with train_writer.as_default():
    tf.summary.scalar('Learning Rate', data = learning_rate, step = epoch)

In [16]:
history=model.fit(
                           train_images,
                           epochs=2,                        # nombre de boucle à réaliser sur le jeu de données complet
                           verbose=1,
                           batch_size = 128 ,
                           callbacks=[CSvlogger,callbacks_list,early_stoping,scheduler_callback]
                           )


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/2
108/108 [==============================] - ETA: 0s - loss: 0.1334 - accuracy: 0.9571

C:\Users\toumi\anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


108/108 [==============================] - 173s 2s/step - loss: 0.1334 - accuracy: 0.9571 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 2/2
108/108 [==============================] - 123s 1s/step - loss: 0.1324 - accuracy: 0.9557 - lr: 0.0010


In [38]:
model.evaluate(test_images)

27/27 [==============================] - 4s 133ms/step - loss: 4.4301 - accuracy: 0.3576


[4.430054664611816, 0.3576388955116272]

In [ ]:
#model.predict(test_images)

In [33]:
history.history

{'loss': [0.1773955374956131,
  0.16065047681331635,
  0.11085831373929977,
  0.11233333498239517,
  0.13462036848068237],
 'accuracy': [0.9412105679512024,
  0.9455546140670776,
  0.962641179561615,
  0.9635099768638611,
  0.9548218846321106]}

In [35]:
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
plt.figure(figsize=(30,30))
for i in range(len(test_images)):
    tt=np.array(test_images[i][0])

    plt.subplot(6,6,i+1)
    tt = tt / 255.0
    #t = np.clip(tt, 0, 1)
    plt.xticks([])
    plt.yticks([])
   #plt.ylabel('Label for Y-axis')
    plt.grid()
    plt.imshow(tt[i])
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['loss'])
plt.ylabel('accuracy')
plt.xlabel('epochs')

plt.legend(['train_acurracy','train_loss'])

In [ ]:
#load csvlogger file and head it
import pandas as pd
dd=pd.read_csv('training.log')
dd.head()